# Word Embeddings in SQLite3

This example uses the SQLite3 Connector within Python3 to store and retrieve various amounts of Word Embeddings as NumPy arrays.

In [1]:
import sqlite3
import io
import os
import time
import numpy
import plotly
import pickle
from tqdm import tqdm_notebook as tqdm

# Dummy Embeddings

For testing purposes we will use randomly generated numpy arrays as dummy embbeddings.

In [2]:
def embeddings(n=10, dim=300):
    """
    Yield n tuples of random numpy arrays of *dim* length indexed by *n*
    """
    idx = 0
    while idx < n:
        yield (str(idx), numpy.random.rand(dim))
        idx += 1

# SQLite Adapders

SQLite only supports a small set of types. To store the NumPy arrays, we will write a custom adapter.

For further details see:
https://docs.python.org/3/library/sqlite3.html#using-adapters-to-store-additional-python-types-in-sqlite-databases

In [3]:
def adapt_array(array):
    """
    Using the numpy.save function to save a binary version of the array,
    and BytesIO to catch the stream of data and convert it into a sqlite3.Binary.
    """
    out = io.BytesIO()
    numpy.save(out, array)
    out.seek(0)

    return sqlite3.Binary(out.read())

def convert_array(blob):
    """
    Using BytesIO to convert the binary version of the array back into a numpy array.
    """
    out = io.BytesIO(blob)
    out.seek(0)

    return numpy.load(out)

# Register the new adapters
sqlite3.register_adapter(numpy.ndarray, adapt_array)
sqlite3.register_converter('array', convert_array)

In [4]:
# Connect to a local database and create a table for the embeddings
connection = sqlite3.connect('./sqlite.embedding.db', detect_types=sqlite3.PARSE_DECLTYPES)
cursor = connection.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS embeddings (key text, embedding array)')
connection.commit()

In [5]:
%%time
# Insert n = 1000 dummy embeddings into the database
for key, emb in embeddings():
    cursor.execute('INSERT INTO embeddings (key, embedding) VALUES (?, ?)', [key, emb])
    connection.commit()

CPU times: user 22.3 ms, sys: 22.9 ms, total: 45.2 ms
Wall time: 1.48 s


In [6]:
%%time
# Select n = 1000 dummy embeddings from the database
for key, _ in embeddings():
    cursor.execute('SELECT * FROM embeddings WHERE key=?', (key,))
    data = cursor.fetchone()
    assert(type(data[1]) is numpy.ndarray)

CPU times: user 23.8 ms, sys: 0 ns, total: 23.8 ms
Wall time: 33.5 ms


In [7]:
# Database size for n = 1000 entries

db_size = os.path.getsize('./sqlite.embedding.db')
print('SQLite database size in MB: {}'.format(db_size / 1024 / 1024))

SQLite database size in MB: 0.046875


# Sample some data

To test the I/O we will write and read some data from the database. This may take a while.

In [8]:
write_times = []
read_times = []
db_sizes = []
counts = [500, 1000, 2000, 3000, 4000, 5000]

for c in counts:
    cursor.execute('DROP TABLE embeddings')
    cursor.execute('CREATE TABLE IF NOT EXISTS embeddings (key text, embedding array)')
    connection.commit()
    
    start_time_write = time.time()
    for key, emb in tqdm(embeddings(c), total=c):
        cursor.execute('INSERT INTO embeddings (key, embedding) VALUES (?, ?)', [key, emb])
        connection.commit()
    write_times.append(time.time() - start_time_write)
    
    start_time_read = time.time()
    for key, emb in embeddings(c):
        cursor.execute('SELECT * FROM embeddings WHERE key=?', (key,))
        data = cursor.fetchone()
    read_times.append(time.time() - start_time_read)
    
    db_sizes.append(os.path.getsize('./sqlite.embedding.db'))   

with open('./collected_times/sqlite_write_times', 'wb') as f:
    pickle.dump(write_times, f)
    
with open('./collected_times/sqlite_read_times', 'wb') as f:
    pickle.dump(read_times, f)
    
print('DONE')


DONE


# Results

In [9]:
# Write Times
plotly.offline.init_notebook_mode(connected=True)
trace = plotly.graph_objs.Scatter(
    x = write_times,
    y = counts,
    mode = 'lines+markers'
)
layout = plotly.graph_objs.Layout(title="SQLite3 Write Times",
                xaxis=dict(title='Time in Seconds'),
                yaxis=dict(title='Embedding Count'))
data = [trace]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='jupyter-basic-scatter')

In [10]:
# Read Times
plotly.offline.init_notebook_mode(connected=True)
trace = plotly.graph_objs.Scatter(
    x = read_times,
    y = counts,
    mode = 'lines+markers'
)
layout = plotly.graph_objs.Layout(title="SQLite3 Read Times",
                xaxis=dict(title='Time in Seconds'),
                yaxis=dict(title='Embedding Count'))
data = [trace]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='jupyter-basic-scatter')

In [11]:
# File Size
plotly.offline.init_notebook_mode(connected=True)
trace = plotly.graph_objs.Bar(
    x = counts,
    y = [size/1024 for size in db_sizes]
)
layout = plotly.graph_objs.Layout(title="Database Size",
                yaxis=dict(title='Size in KB'),
                xaxis=dict(title='Embedding Count'))
data = [trace]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='jupyter-basic_bar')